In [3]:
import pytest
import re

import torch
from torch.utils.data import DataLoader
import torchvision

import modlee
from modlee import data_metafeatures as dmf
from modlee.utils import text_loaders

import spacy

IMAGE_DATALOADER = modlee.utils.get_imagenette_dataloader()
# TEXT_DATALOADER = modlee.utils.get_wnli_dataloader() 


TEXT_LOADERS = {loader_fn:getattr(text_loaders, loader_fn) for loader_fn in dir(text_loaders) if re.match('get_(.*)_dataloader', loader_fn)}


/home/ubuntu/.conda/envs/modlee3.10/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


100%|██████████| 99003388/99003388 [00:01<00:00, 54709695.13it/s]


Extracting .data/imagenette2-160.tgz to .data


In [47]:
import pandas as pd
df = None
features = []
labels = []
for text_loader_name,text_loader_fn in TEXT_LOADERS.items():
    print(text_loader_name)
    for _ in range(10):
        text_mf = dmf.TextDataMetafeatures(text_loader_fn(), testing=True)
        features.append({
            **text_mf.embedding,
            **text_mf.mfe,
            **text_mf.properties
        })
        labels.append(text_loader_name)
        # features.append(text_mf.)

get_cola_dataloader
get_mnli_dataloader
get_qnli_dataloader
get_rte_dataloader
get_sst2_dataloader
get_stsb_dataloader
get_wnli_dataloader


In [48]:
df = pd.DataFrame(features)
# print(len(TEXT_LOADERS))
df = df.fillna(0)

In [49]:
# print(df.dtypes)
import numpy as np
object_columns = df.select_dtypes(include=['object']).columns
df[object_columns] = df[object_columns].apply(
    lambda x : x[0]
)
df.to_csv('text_metafeatures.tsv', sep='\t', index=False, header=False)

In [53]:
def min_max_normalize(column):
    return (column - column.min()) / (column.max() - column.min())

# Normalize DataFrame by columns
normalized_df = df.apply(min_max_normalize)
normalized_df.to_csv('text_metafeatures_normalized.tsv', sep='\t', index=False, header=False)
with open("data_labels.txt",'w') as _file:
    _file.write('\n'.join(labels))
    # _file.write('\n'.join(list(TEXT_LOADERS.keys())))

In [51]:
!code ./text_metafeatures.tsv

In [52]:
print(list(TEXT_LOADERS.keys()), sep='\n')

['get_cola_dataloader', 'get_mnli_dataloader', 'get_qnli_dataloader', 'get_rte_dataloader', 'get_sst2_dataloader', 'get_stsb_dataloader', 'get_wnli_dataloader']
